# <center> Homework </center>
---

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

---

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "__Green__ Taxi Trip Records", we'll use "__Yellow__ Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?
- 16
- 17
- 18
- 19

In [2]:
jan_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet'
feb_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet'

## Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?
- 46.45
- 41.45
- 56.45
- 51.45

In [ ]:
pass

## Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?
- 90%
- 92%
- 95%
- 98%

In [ ]:
pass

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
- Turn the dataframe into a list of dictionaries
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?
- 2
- 155
- 345
- 515
- 715

In [ ]:
pass

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.
- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?
- 6.99
- 11.99
- 16.99
- 21.99

In [ ]:
pass

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?
- 7.79
- 12.79
- 17.79
- 22.79

In [ ]:
pass

## Submit the results

- Submit your results here: https://forms.gle/uYTnWrcsubi2gdGV7
- You can submit your solution multiple times. In this case, only the last submission will be used
- If your answer doesn't match options exactly, select the closest one

## Deadline

The deadline for submitting is 23 May 2023 (Tuesday), 23:00 CEST (Berlin time).

After that, the form will be closed.